In [6]:
array_ts = []
array_ts.append({1:123})
print(array_ts)
array_ts.append({2:456})
print(array_ts)

[{1: 123}]
[{1: 123}, {2: 456}]


In [22]:
d1 = float('-inf')
d2 = float('-inf')
if d1 == d2 :
    print("yes")

yes


In [20]:
test1 = [1, 1, 2, 3, 3, 4]
test1 = list(set(test1))
test1

[1, 2, 3, 4]